In [1]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv("/Users/gauravjangid/PycharmProjects/Insurance Project/Dataset/insurance_processed.csv")


In [5]:
from sklearn.model_selection import train_test_split
x=df.drop(columns=['charges'])
y=df['charges']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [7]:
df.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [8]:
preprocessor=ColumnTransformer(transformers=[
('tnf1',OneHotEncoder(handle_unknown='ignore'),['sex','smoker','region']),
    ('tnf4',StandardScaler(),['age','bmi'])
])

In [9]:
from sklearn.tree import DecisionTreeRegressor
pipeline=Pipeline(steps=[
    ('transformer',preprocessor),
    ('dt_regression',DecisionTreeRegressor())
])


In [10]:
model=pipeline.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [11]:
from sklearn.metrics import r2_score
r2_score(y_pred,y_test)*100

71.63538548590293

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
ls=[]
for model_train in [LinearRegression,DecisionTreeRegressor,RandomForestRegressor]:
    pipeline=Pipeline(steps=[
    ('transformer',preprocessor),
    ('model',model_train())])
    
    model=pipeline.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    ls.append(model)
    R2_score=r2_score(y_pred,y_test)*100
    print(f'Accuracy {R2_score} {model_train.__name__} ')
    

Accuracy 69.14689625892613 LinearRegression 
Accuracy 70.35395268978566 DecisionTreeRegressor 
Accuracy 82.50903442325941 RandomForestRegressor 


In [13]:
#hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV

pipeline=ls[2]

param_dist = {
    'model__n_estimators': [50, 100, 200, 300],  # Number of trees
    'model__max_depth': [10, 20, 30, None],  # Depth of trees
    'model__min_samples_split': [2, 5, 10],  # Minimum samples to split
    'model__min_samples_leaf': [1, 2, 4],  # Minimum samples in leaf
}

random_search = RandomizedSearchCV(
    pipeline, param_distributions=param_dist, 
    n_iter=10, cv=5, scoring='r2', n_jobs=-1, random_state=42
)

random_search.fit(x_train, y_train)
best_model = random_search.best_estimator_
y_pred = best_model.predict(x_test)
r2_score(y_pred,y_test)*100


84.71857577269671

In [14]:
import pickle

with open('best_model.pkl', 'wb') as file:
    pickle.dump(pipeline, file)


In [15]:
data=pd.DataFrame([[20,"male",22,0,"yes",'northwest']],columns=['age', 'sex', 'bmi', 'children', 'smoker', 'region'])
pipeline.predict(data)

array([15810.760205])

In [16]:
data=x_test[:1]
pipeline.predict(data)

array([8872.777471])

In [17]:
data


,age,sex,bmi,children,smoker,region
764,45,female,25.175,2,no,northeast


In [18]:
import sklearn
print(sklearn.__version__)


1.6.1


In [19]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [20]:
df['region'].unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

In [188]:
data=pd.DataFrame([[31,"male",32,1,"yes",'southwest']],columns=['age', 'sex', 'bmi', 'children', 'smoker', 'region'])
pipeline.predict(data)

array([41934.5910528])